In [2]:
import pandas as pd
import pyodbc
connection_string = ("DRIVER={SQL Server};"
"SERVER=USDF11V0954;"
"DATABASE=Actuarial_AH;"
"Trust_Connection=yes;")

conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [3]:
med = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
    WHERE DRUG_NAME_PREFERRED IN ('Wegovy', 'Saxenda', 'Zepbound')
    AND SERVICE_DATE >= '2022-01-01'
    AND SERVICE_DATE < '2025-01-01'
)
SELECT M.MEDICAL_CLAIM_ID, 
       M.MEMBER_ID, 
       M.PAID_DATE, 
	   YEAR(M.PAID_DATE) AS PAID_YEAR,
       M.SERVICE_DATE,
YEAR(M.SERVICE_DATE) AS SERVICE_YEAR,
       M.MEDICAL_PAID_AMOUNT, 
       M.IS_TELEMEDICINE, 
       M.IS_ER_AVOIDABLE, 
M.DIAGNOSIS_DESC_ICD10_1 AS PRIMARY_DX,
       M.PROCEDURE_DESC, 
       M.ARTTOS_V2_L1, 
       M.ARTTOS_V2_L3, 
M.DX_IS_CHRONIC,
       M.ICD10_CHAPTER, 
       M.ICD10_CATEGORY, 
       M.ICD10_SECTION, 
       M.MEG_EPISODE_DESCRIPTION, 
       M.EMPLOYER_NAME, 
       M.OP_SURG_INC,
       M.IS_PCP_VISIT, 
       M.INCLUDED_SPECIALIST, 
       M.ER_VISIT_FLAG, 
       M.IS_URGENT_CARE_VISIT, 
       M.IS_PREVENTIVE_VISIT, 
       M.IP_ADMIT_INC
FROM [Actuarial_AH].[dbo].[SN_Medical] M
JOIN 
Actuarial_AH.dbo.SN_Member MB ON M.MEMBER_ID = MB.MEMBER_ID
WHERE M.PAID_DATE BETWEEN '2021-01-01' AND '2024-12-31'
AND MB.MONTH_KEY = '2024-12-01' 
AND MB.MEDICAL_ENROLLMENT_STATUS = 3
AND MB.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND MB.MEMBER_ID IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
    HAVING COUNT(DISTINCT PRESCRIPTION_YEAR) = 3
	)
"""
med = pd.read_sql(med, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_24324\1269032955.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  med = pd.read_sql(med, conn)


In [4]:
rx = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
    WHERE DRUG_NAME_PREFERRED IN ('Wegovy', 'Saxenda', 'Zepbound')
    AND SERVICE_DATE BETWEEN '2022-01-01' AND '2024-12-31'
)
SELECT R.RX_CLAIM_ID, 
       R.MEMBER_ID, 
       R.PAID_DATE, 
       R.SERVICE_DATE, 
       R.DRUG_NAME_GENERIC,
	   R.DRUG_NAME_PREFERRED,
	   R.RX_PAID_AMOUNT,
	   R.RX_SCRIPT_COUNT,
	   R.ART_DRUGGRPS_L1,
	   R.ART_DRUGGRPS_L2,
	   YEAR(R.PAID_DATE) AS PAID_YEAR,
YEAR(R.SERVICE_DATE) AS SERVICE_YEAR
FROM [Actuarial_AH].[dbo].[SN_Rx] R
JOIN 
Actuarial_AH.dbo.SN_Member M ON R.MEMBER_ID = M.MEMBER_ID
WHERE R.PAID_DATE BETWEEN '2021-01-01' AND '2024-12-31'
AND M.MONTH_KEY = '2024-12-01' 
AND M.MEDICAL_ENROLLMENT_STATUS = 3
AND M.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND M.MEMBER_ID IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
    HAVING COUNT(DISTINCT PRESCRIPTION_YEAR) = 3
	)
"""
rx = pd.read_sql(rx, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_24324\2711416039.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rx = pd.read_sql(rx, conn)


In [5]:
member = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
    WHERE DRUG_NAME_PREFERRED IN ('Wegovy', 'Saxenda', 'Zepbound')
    AND SERVICE_DATE >= '2022-01-01'
    AND SERVICE_DATE < '2025-01-01'
)
SELECT M.MEMBER_ID,
       M.MEMBER_STATUS,
       M.EMPLOYER_NAME,
       M.HOME_STATE,
	   CASE 
    WHEN M.MEMBER_RELATIONSHIP = 0 THEN 'Subscriber'
    WHEN M.MEMBER_RELATIONSHIP = 1 THEN 'Spouse'
    WHEN M.MEMBER_RELATIONSHIP = 2 THEN 'Dependent'
    WHEN M.MEMBER_RELATIONSHIP = 3 THEN 'Domestic Partner'
    WHEN M.MEMBER_RELATIONSHIP = 4 THEN 'Unknown'
    ELSE 'Unknown' -- Optional: Handle unexpected values
END AS MEMBER_RELATIONSHIP,
CASE 
        WHEN M.GENDER = 0 THEN 'Female'
        WHEN M.GENDER = 1 THEN 'Male'
        WHEN M.GENDER = 2 THEN 'Unknown'
        ELSE 'Unknown' 
    END AS GENDER,
    DATEDIFF(YEAR, CONVERT(DATE, M.BIRTH_YEAR_MO + '-01'), GETDATE()) - 
    CASE 
        WHEN MONTH(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) > MONTH(GETDATE()) 
             OR (MONTH(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) = MONTH(GETDATE()) 
                 AND DAY(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) > DAY(GETDATE())) 
        THEN 1 
        ELSE 0 
    END AS Age,
	CASE 
    WHEN M.BIRTH_YEAR >= 1946 AND M.BIRTH_YEAR < 1965 THEN 'Baby Boomers'
	WHEN M.BIRTH_YEAR >= 1965 AND M.BIRTH_YEAR < 1981 THEN 'Generation X'
	WHEN M.BIRTH_YEAR >= 1981 AND M.BIRTH_YEAR < 1997 THEN 'Millenials'
	WHEN M.BIRTH_YEAR >= 1997 AND M.BIRTH_YEAR < 2012 THEN 'Generation Z'
    ELSE 'Unknown' -- Optional: Handle unexpected values
END AS GENERATIONS
FROM [Actuarial_AH].[dbo].[SN_MEMBER] M
WHERE M.MEMBER_ID IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
    HAVING COUNT(DISTINCT PRESCRIPTION_YEAR) = 3
	)
AND M.MONTH_KEY = '2024-12-01' 
AND M.MEDICAL_ENROLLMENT_STATUS = 3
AND M.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'


"""
member = pd.read_sql(member, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_24324\1763390720.py:56: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  member = pd.read_sql(member, conn)


In [6]:
med

,MEDICAL_CLAIM_ID,MEMBER_ID,PAID_DATE,PAID_YEAR,SERVICE_DATE,SERVICE_YEAR,MEDICAL_PAID_AMOUNT,IS_TELEMEDICINE,IS_ER_AVOIDABLE,PRIMARY_DX,...,ICD10_SECTION,MEG_EPISODE_DESCRIPTION,EMPLOYER_NAME,OP_SURG_INC,IS_PCP_VISIT,INCLUDED_SPECIALIST,ER_VISIT_FLAG,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,7913181.0,mma-e29d6c47fcc4433a97ed373b239be2f7,2021-01-01,2021,2020-12-09,2020,0.00,0.0,0.0,Incisional hernia without obstruction or gangrene,...,Ventral hernia,"Hernia, External",CHOP,1,0,0,0,0,0,0
1,951883.0,mma-5900ccfe10e042f084a2a4db5170f0bd,2021-01-01,2021,2020-12-23,2020,5.50,0.0,1.0,Essential (primary) hypertension,...,Essential (primary) hypertension,"Essential Hypertension, Chronic Maintenance",BUTTERBALL,0,0,0,0,0,0,0
2,7913181.0,mma-e29d6c47fcc4433a97ed373b239be2f7,2021-01-01,2021,2020-12-09,2020,0.00,0.0,0.0,Incisional hernia without obstruction or gangrene,...,Ventral hernia,"Hernia, External",CHOP,1,0,0,0,0,0,0
3,7913181.0,mma-e29d6c47fcc4433a97ed373b239be2f7,2021-01-01,2021,2020-12-09,2020,0.00,0.0,0.0,Incisional hernia without obstruction or gangrene,...,Ventral hernia,"Hernia, External",CHOP,1,0,0,0,0,0,0
4,7913181.0,mma-e29d6c47fcc4433a97ed373b239be2f7,2021-01-01,2021,2020-12-09,2020,8284.95,0.0,0.0,Incisional hernia without obstruction or gangrene,...,Ventral hernia,"Hernia, External",CHOP,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187022,7267041.0,mma-95854ec3b2ad49f6b7c782bab6e175ee,2024-12-31,2024,2024-12-11,2024,0.00,0.0,1.0,Pain in right hip,...,"Other joint disorder, not elsewhere classified","Fracture, Dislocation, or Sprain: Hip or Pelvis",CRETEX,0,0,0,0,0,0,0
187023,3066260.0,mma-95854ec3b2ad49f6b7c782bab6e175ee,2024-12-31,2024,2024-11-26,2024,0.00,0.0,1.0,Other low back pain,...,Dorsalgia,Intervertebral Disc Disorders: Lumbar and Lumb...,CRETEX,0,0,0,0,0,0,0
187024,3066260.0,mma-95854ec3b2ad49f6b7c782bab6e175ee,2024-12-31,2024,2024-11-26,2024,31.81,0.0,1.0,Pain in left hip,...,"Other joint disorder, not elsewhere classified","Fracture, Dislocation, or Sprain: Hip or Pelvis",CRETEX,0,0,0,0,0,0,0
187025,3066260.0,mma-95854ec3b2ad49f6b7c782bab6e175ee,2024-12-31,2024,2024-11-26,2024,22.99,0.0,1.0,Pain in right hip,...,"Other joint disorder, not elsewhere classified","Fracture, Dislocation, or Sprain: Hip or Pelvis",CRETEX,0,0,0,0,0,0,0


In [7]:
rx

,RX_CLAIM_ID,MEMBER_ID,PAID_DATE,SERVICE_DATE,DRUG_NAME_GENERIC,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT,RX_SCRIPT_COUNT,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,PAID_YEAR,SERVICE_YEAR
0,12297591.0,mma-1f4e97ba2e484b67a0efeacfe9da5830,2021-01-04,2021-01-04,Simvastatin,Simvastatin,7.12,90.0,Lipid Agents,HMG CoA Reductase Inhibitors,2021,2021
1,14273997.0,mma-606ed05cd58a4e50848b2f5283e3050d,2021-01-04,2021-01-04,Montelukast Sodium,Montelukast Sodium,5.58,30.0,Respiratory,Leukotriene Inhibitors,2021,2021
2,9814135.0,mma-0f3528bd0a8843ee84bf4bc4940847a9,2021-01-01,2020-12-30,Montelukast Sodium,Montelukast Sodium,32.90,90.0,Respiratory,Leukotriene Inhibitors,2021,2020
3,2485661.0,mma-45c7f7c76db040efb815a825839037eb,2021-01-01,2020-12-26,Latanoprost,Latanoprost,7.60,2.5,Opthalmic,Glaucoma Agents,2021,2020
4,6122478.0,mma-449c02fb4898483b8bf638b4f65b8f49,2021-01-05,2021-01-05,Propranolol HCl ER,Propranolol HCl ER,72.09,30.0,Cardiac,Beta-Blockers,2021,2021
...,...,...,...,...,...,...,...,...,...,...,...,...
116444,15933024.0,mma-8a64fddd24cc4e7f9e5043394aa4fc55,2024-12-31,2024-12-31,Tirzepatide-Weight Management,Zepbound,966.56,2.0,CNS Agents,Weight Loss Agents,2024,2024
116445,656275.0,mma-d9fa4b33e14b458992bf0617961ac987,2024-12-31,2024-12-31,amLODIPine Besylate,amLODIPine Besylate,24.18,90.0,Cardiac,Calcium Channel Blockers,2024,2024
116446,17317999.0,mma-b5db1ec463b84d3f83a39390fef8d18c,2024-12-31,2024-12-31,Scopolamine,Scopolamine,46.59,4.0,Gastrointestinal,Antiemetics,2024,2024
116447,1338764.0,mma-c6dd6e584c00480dad1d167a23e430e5,2024-12-31,2024-12-31,ALPRAZolam,ALPRAZolam,2.76,30.0,CNS Agents,Benzodiazepines,2024,2024


In [8]:
member

,MEMBER_ID,MEMBER_STATUS,EMPLOYER_NAME,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS
0,mma-5a400fb9c1934bcf87c25301710034c3,0.0,SELECT MEDICAL,PA,Spouse,Female,57,Generation X
1,mma-2e25ed887e0b4990ad2fa1c2d91e0966,0.0,WAKE COUNTY GOVERNMENT,NC,Subscriber,Female,53,Generation X
2,mma-b027fbc0d58348338e0a12b5520743a1,0.0,WAKEMED,NC,Subscriber,Female,48,Generation X
3,mma-8f71aebcee534b149fcb3a24f53d271c,0.0,BUTTERBALL,NC,Subscriber,Female,45,Generation X
4,mma-6bdc0ffa0d8e406c8d5c977b9573bc60,0.0,DALLAS AREA RAPID TRANSIT,TX,Subscriber,Female,40,Millenials
...,...,...,...,...,...,...,...,...
721,mma-bbe87a52bbd64d458e04d757809d8531,0.0,SELECT MEDICAL,CT,Subscriber,Female,31,Millenials
722,mma-8bb9aad32c0741fc9e75241e57690ba9,0.0,TOWN OF APEX,NC,Spouse,Female,42,Millenials
723,mma-492971da62b4438e858c9d296391bd0b,0.0,WAKE COUNTY GOVERNMENT,NC,Subscriber,Female,63,Baby Boomers
724,mma-cf3653bd825d45f283ec00446623fa0d,0.0,SELECT MEDICAL,TN,Subscriber,Female,60,Generation X


In [9]:
# Define the drugs to filter out
drugs_to_remove = ['Wegovy', 'Saxenda', 'Zepbound']

# Filter to find members with the specified drugs in 2021
members_to_remove = rx[(rx['SERVICE_YEAR'] == 2021) & (rx['DRUG_NAME_PREFERRED'].isin(drugs_to_remove))]['MEMBER_ID'].unique()

# Remove those members from the original DataFrame
rx_members_glp_22_24 = rx[~rx['MEMBER_ID'].isin(members_to_remove)]
med_members_glp_22_24 = med[~med['MEMBER_ID'].isin(members_to_remove)]
members_glp_22_24 = member[~member['MEMBER_ID'].isin(members_to_remove)]

In [10]:
med_members_glp_22_24

,MEDICAL_CLAIM_ID,MEMBER_ID,PAID_DATE,PAID_YEAR,SERVICE_DATE,SERVICE_YEAR,MEDICAL_PAID_AMOUNT,IS_TELEMEDICINE,IS_ER_AVOIDABLE,PRIMARY_DX,...,ICD10_SECTION,MEG_EPISODE_DESCRIPTION,EMPLOYER_NAME,OP_SURG_INC,IS_PCP_VISIT,INCLUDED_SPECIALIST,ER_VISIT_FLAG,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
7,11930232.0,mma-058096e5855548fc969a075fad768bd5,2021-01-01,2021,2020-12-14,2020,49.54,0.0,0.0,Neoplasm of uncertain behavior of skin,...,Neoplasm of uncertain behavior of other and un...,Neoplasm: Atypical Nevus,CHOP,0,0,0,0,0,0,0
11,5120382.0,mma-d46e53c84adb49c8b8f735f88919b4c8,2021-01-01,2021,2020-09-30,2020,0.00,0.0,1.0,Encounter for gynecological examination (gener...,...,Encounter for other special examination withou...,Encounter for Preventive Health Services,CHOP,0,0,0,0,0,1,0
17,5120382.0,mma-d46e53c84adb49c8b8f735f88919b4c8,2021-01-01,2021,2020-09-30,2020,-126.50,0.0,1.0,Encounter for gynecological examination (gener...,...,Encounter for other special examination withou...,Encounter for Preventive Health Services,CHOP,0,0,0,0,0,1,0
21,18638487.0,mma-60fcb98621904c5ab8068626e87e6d92,2021-01-01,2021,2020-12-22,2020,0.00,0.0,0.0,"Myalgia, other site",...,"Other and unspecified soft tissue disorders, n...","Other Arthropathies, Bone and Joint Disorders",TOKIO MARINE NORTH AMERICA,1,0,0,0,0,0,0
22,6612015.0,mma-60fcb98621904c5ab8068626e87e6d92,2021-01-01,2021,2020-12-17,2020,0.00,0.0,0.0,"Alcohol dependence, uncomplicated",...,Alcohol related disorders,"Drug Abuse, Dependence, Intoxication: Alcohol",TOKIO MARINE NORTH AMERICA,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187022,7267041.0,mma-95854ec3b2ad49f6b7c782bab6e175ee,2024-12-31,2024,2024-12-11,2024,0.00,0.0,1.0,Pain in right hip,...,"Other joint disorder, not elsewhere classified","Fracture, Dislocation, or Sprain: Hip or Pelvis",CRETEX,0,0,0,0,0,0,0
187023,3066260.0,mma-95854ec3b2ad49f6b7c782bab6e175ee,2024-12-31,2024,2024-11-26,2024,0.00,0.0,1.0,Other low back pain,...,Dorsalgia,Intervertebral Disc Disorders: Lumbar and Lumb...,CRETEX,0,0,0,0,0,0,0
187024,3066260.0,mma-95854ec3b2ad49f6b7c782bab6e175ee,2024-12-31,2024,2024-11-26,2024,31.81,0.0,1.0,Pain in left hip,...,"Other joint disorder, not elsewhere classified","Fracture, Dislocation, or Sprain: Hip or Pelvis",CRETEX,0,0,0,0,0,0,0
187025,3066260.0,mma-95854ec3b2ad49f6b7c782bab6e175ee,2024-12-31,2024,2024-11-26,2024,22.99,0.0,1.0,Pain in right hip,...,"Other joint disorder, not elsewhere classified","Fracture, Dislocation, or Sprain: Hip or Pelvis",CRETEX,0,0,0,0,0,0,0


In [11]:
med_agg = med_members_glp_22_24.pivot_table(index='MEMBER_ID', columns='PAID_YEAR', values='MEDICAL_PAID_AMOUNT', aggfunc='sum')
med_agg.reset_index(inplace=True)
med_agg.columns.name = None
med_agg.columns = [f"{col}" if col != 'PAID_YEAR' else 'PAID_YEAR' for col in med_agg.columns]
med_agg.rename(columns={'2021': 'Med_2021', '2022': 'Med_2022', '2023': 'Med_2023', '2024': 'Med_2024'}, inplace=True)
med_agg1 = med_agg
med_agg1

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,708.74,1069.66,13211.02,7120.77
1,mma-0054034112c64cd2924c4f85de9a46ca,8338.34,739.25,780.80,2225.00
2,mma-010b8797194145fbb3687e60959516b1,2493.20,621.00,79.93,734.63
3,mma-01c846aa858d43e98167eb0669db3a4e,951.22,496.56,449.34,330.20
4,mma-0281be178d4042ab83dd30b3190a3c6c,3834.97,2604.28,3228.46,14857.68
...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,43039.79,64591.43,32627.37,35547.06
457,mma-fdc88c2f941e45d3808795b5eb01e19e,1687.41,1501.48,1604.11,2188.31
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,45.33,28.19,0.00,204.80
459,mma-ff82ab56bc48478b91ad9760b62d56e1,11481.20,2310.52,1079.37,14244.44


In [12]:

# Define inflation rates (as factors)
inflation_factors = {
    2021: 1.175, 
    2022: 1.117,   
    2023: 1.069, 
    2024: 1
}

# Normalize employer paid amounts for inflation
for year, factor in inflation_factors.items():
    column_name = f'Med_{year}'
    med_agg[column_name] = med_agg[column_name] * factor
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,832.76950,1194.81022,14122.58038,7120.77
1,mma-0054034112c64cd2924c4f85de9a46ca,9797.54950,825.74225,834.67520,2225.00
2,mma-010b8797194145fbb3687e60959516b1,2929.51000,693.65700,85.44517,734.63
3,mma-01c846aa858d43e98167eb0669db3a4e,1117.68350,554.65752,480.34446,330.20
4,mma-0281be178d4042ab83dd30b3190a3c6c,4506.08975,2908.98076,3451.22374,14857.68
...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,50571.75325,72148.62731,34878.65853,35547.06
457,mma-fdc88c2f941e45d3808795b5eb01e19e,1982.70675,1677.15316,1714.79359,2188.31
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,53.26275,31.48823,0.00000,204.80
459,mma-ff82ab56bc48478b91ad9760b62d56e1,13490.41000,2580.85084,1153.84653,14244.44


In [13]:
med_agg['Med_Delta'] = med_agg['Med_2024'] - med_agg['Med_2021']
med_agg['Med_%_Change'] =(( med_agg['Med_2024'] - med_agg['Med_2021'])/ med_agg['Med_2021']) * 100
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,832.76950,1194.81022,14122.58038,7120.77,6288.00050,755.070941
1,mma-0054034112c64cd2924c4f85de9a46ca,9797.54950,825.74225,834.67520,2225.00,-7572.54950,-77.290240
2,mma-010b8797194145fbb3687e60959516b1,2929.51000,693.65700,85.44517,734.63,-2194.88000,-74.923110
3,mma-01c846aa858d43e98167eb0669db3a4e,1117.68350,554.65752,480.34446,330.20,-787.48350,-70.456753
4,mma-0281be178d4042ab83dd30b3190a3c6c,4506.08975,2908.98076,3451.22374,14857.68,10351.59025,229.724458
...,...,...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,50571.75325,72148.62731,34878.65853,35547.06,-15024.69325,-29.709655
457,mma-fdc88c2f941e45d3808795b5eb01e19e,1982.70675,1677.15316,1714.79359,2188.31,205.60325,10.369827
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,53.26275,31.48823,0.00000,204.80,151.53725,284.508873
459,mma-ff82ab56bc48478b91ad9760b62d56e1,13490.41000,2580.85084,1153.84653,14244.44,754.03000,5.589378


In [14]:
med_agg.fillna(0, inplace=True)

In [15]:
med_agg.isnull().sum()

MEMBER_ID       0
Med_2021        0
Med_2022        0
Med_2023        0
Med_2024        0
Med_Delta       0
Med_%_Change    0
dtype: int64

In [16]:
import numpy as np

# Prepare the years and medical costs
years = np.array([2021, 2022, 2023, 2024])
slope_list = []

# Calculate the slope for each member
for index, row in med_agg.iterrows():
    costs = np.array([row['Med_2021'], row['Med_2022'], row['Med_2023'], row['Med_2024']])
    # Perform linear regression to get the slope
    slope, intercept = np.polyfit(years, costs, 1)  # 1 indicates linear fit
    slope_list.append(slope)

# Add the slope to the DataFrame
med_agg['Med_Slope'] = slope_list
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,832.76950,1194.81022,14122.58038,7120.77,6288.00050,755.070941,3179.177166
1,mma-0054034112c64cd2924c4f85de9a46ca,9797.54950,825.74225,834.67520,2225.00,-7572.54950,-77.290240,-2270.871555
2,mma-010b8797194145fbb3687e60959516b1,2929.51000,693.65700,85.44517,734.63,-2194.88000,-74.923110,-719.285183
3,mma-01c846aa858d43e98167eb0669db3a4e,1117.68350,554.65752,480.34446,330.20,-787.48350,-70.456753,-243.676356
4,mma-0281be178d4042ab83dd30b3190a3c6c,4506.08975,2908.98076,3451.22374,14857.68,10351.59025,229.724458,3159.701373
...,...,...,...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,50571.75325,72148.62731,34878.65853,35547.06,-15024.69325,-29.709655,-8234.404853
457,mma-fdc88c2f941e45d3808795b5eb01e19e,1982.70675,1677.15316,1714.79359,2188.31,205.60325,10.369827,65.445018
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,53.26275,31.48823,0.00000,204.80,151.53725,284.508873,42.312352
459,mma-ff82ab56bc48478b91ad9760b62d56e1,13490.41000,2580.85084,1153.84653,14244.44,754.03000,5.589378,83.508569


In [17]:
rx_agg = rx_members_glp_22_24.pivot_table(index='MEMBER_ID', columns='PAID_YEAR', values='RX_PAID_AMOUNT', aggfunc='sum')
rx_agg.reset_index(inplace=True)
rx_agg.columns.name = None
rx_agg.columns = [f"{col}" if col != 'PAID_YEAR' else 'PAID_YEAR' for col in rx_agg.columns]
rx_agg.rename(columns={'2021': 'Rx_2021', '2022': 'Rx_2022', '2023': 'Rx_2023', '2024': 'Rx_2024'}, inplace=True)
rx_agg1 = rx_agg
rx_agg1

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,7596.54,15354.04,20686.86,13584.34
1,mma-0054034112c64cd2924c4f85de9a46ca,NaN,14183.54,14311.33,18501.11
2,mma-010b8797194145fbb3687e60959516b1,12.74,3812.79,3781.08,6361.50
3,mma-01c846aa858d43e98167eb0669db3a4e,1809.09,13926.68,9073.36,5200.05
4,mma-0281be178d4042ab83dd30b3190a3c6c,791.36,5048.15,15017.36,17215.45
...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,2679.97,6276.88,24392.79,15549.13
457,mma-fdc88c2f941e45d3808795b5eb01e19e,114.09,6248.18,15425.84,18245.69
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,3082.47,10946.63,19182.19,18446.81
459,mma-ff82ab56bc48478b91ad9760b62d56e1,321.53,3053.49,1877.35,11416.44


In [18]:
# Define inflation rates (as factors)
rx_inflation_factors = {
    2021: 1.31, 
    2022: 1.20,   
    2023: 1.103, 
    2024: 1
}

# Normalize employer paid amounts for inflation
for year, factor in rx_inflation_factors.items():
    column_name = f'Rx_{year}'
    rx_agg[column_name] = rx_agg[column_name] * factor
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,9951.4674,18424.848,22817.60658,13584.34
1,mma-0054034112c64cd2924c4f85de9a46ca,NaN,17020.248,15785.39699,18501.11
2,mma-010b8797194145fbb3687e60959516b1,16.6894,4575.348,4170.53124,6361.50
3,mma-01c846aa858d43e98167eb0669db3a4e,2369.9079,16712.016,10007.91608,5200.05
4,mma-0281be178d4042ab83dd30b3190a3c6c,1036.6816,6057.780,16564.14808,17215.45
...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,3510.7607,7532.256,26905.24737,15549.13
457,mma-fdc88c2f941e45d3808795b5eb01e19e,149.4579,7497.816,17014.70152,18245.69
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,4038.0357,13135.956,21157.95557,18446.81
459,mma-ff82ab56bc48478b91ad9760b62d56e1,421.2043,3664.188,2070.71705,11416.44


In [19]:
rx_agg.fillna(0, inplace=True)
rx_agg.isnull().sum()

MEMBER_ID    0
Rx_2021      0
Rx_2022      0
Rx_2023      0
Rx_2024      0
dtype: int64

In [20]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [21]:
rx_agg['Rx_Delta'] = rx_agg['Rx_2024'] - rx_agg['Rx_2021']
rx_agg['Rx_%_Change'] =((rx_agg['Rx_2024'] - rx_agg['Rx_2021'])/ rx_agg['Rx_2021']) * 100
# Prepare the years and medical costs
years = np.array([2021, 2022, 2023, 2024])
slope_list = []

# Calculate the slope for each member
for index, row in rx_agg.iterrows():
    costs = np.array([row['Rx_2021'], row['Rx_2022'], row['Rx_2023'], row['Rx_2024']])
    # Perform linear regression to get the slope
    slope, intercept = np.polyfit(years, costs, 1)  # 1 indicates linear fit
    slope_list.append(slope)

# Add the slope to the DataFrame
rx_agg['Rx_Slope'] = slope_list
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,9951.47,18424.85,22817.61,13584.34,3632.87,36.51,1529.14
1,mma-0054034112c64cd2924c4f85de9a46ca,0.00,17020.25,15785.40,18501.11,18501.11,inf,5426.85
2,mma-010b8797194145fbb3687e60959516b1,16.69,4575.35,4170.53,6361.50,6344.81,38017.01,1862.96
3,mma-01c846aa858d43e98167eb0669db3a4e,2369.91,16712.02,10007.92,5200.05,2830.14,119.42,178.63
4,mma-0281be178d4042ab83dd30b3190a3c6c,1036.68,6057.78,16564.15,17215.45,16178.77,1560.63,5904.27
...,...,...,...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,3510.76,7532.26,26905.25,15549.13,12038.37,342.90,5548.81
457,mma-fdc88c2f941e45d3808795b5eb01e19e,149.46,7497.82,17014.70,18245.69,18096.23,12107.91,6380.56
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,4038.04,13135.96,21157.96,18446.81,14408.77,356.83,5124.83
459,mma-ff82ab56bc48478b91ad9760b62d56e1,421.20,3664.19,2070.72,11416.44,10995.24,2610.43,3139.22


In [22]:
rx_agg.replace(np.inf, 0, inplace=True)
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,9951.47,18424.85,22817.61,13584.34,3632.87,36.51,1529.14
1,mma-0054034112c64cd2924c4f85de9a46ca,0.00,17020.25,15785.40,18501.11,18501.11,0.00,5426.85
2,mma-010b8797194145fbb3687e60959516b1,16.69,4575.35,4170.53,6361.50,6344.81,38017.01,1862.96
3,mma-01c846aa858d43e98167eb0669db3a4e,2369.91,16712.02,10007.92,5200.05,2830.14,119.42,178.63
4,mma-0281be178d4042ab83dd30b3190a3c6c,1036.68,6057.78,16564.15,17215.45,16178.77,1560.63,5904.27
...,...,...,...,...,...,...,...,...
456,mma-fd3a666933464a569791e983fbdb4867,3510.76,7532.26,26905.25,15549.13,12038.37,342.90,5548.81
457,mma-fdc88c2f941e45d3808795b5eb01e19e,149.46,7497.82,17014.70,18245.69,18096.23,12107.91,6380.56
458,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,4038.04,13135.96,21157.96,18446.81,14408.77,356.83,5124.83
459,mma-ff82ab56bc48478b91ad9760b62d56e1,421.20,3664.19,2070.72,11416.44,10995.24,2610.43,3139.22


In [23]:
member_mrg = members_glp_22_24.merge(med_agg, on='MEMBER_ID')
member_mrg = member_mrg.merge(rx_agg, on='MEMBER_ID')                                     
member_mrg                                    

,MEMBER_ID,MEMBER_STATUS,EMPLOYER_NAME,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,...,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-5a400fb9c1934bcf87c25301710034c3,0.00,SELECT MEDICAL,PA,Spouse,Female,57,Generation X,5783.55,11169.72,...,7882.82,136.30,2346.32,63018.44,91263.11,52811.56,69490.32,6471.88,10.27,-1903.59
1,mma-2e25ed887e0b4990ad2fa1c2d91e0966,0.00,WAKE COUNTY GOVERNMENT,NC,Subscriber,Female,53,Generation X,32448.94,4684.03,...,-25683.37,-79.15,-6880.27,1063.89,8594.32,19333.53,13359.87,12295.98,1155.76,4762.72
2,mma-b027fbc0d58348338e0a12b5520743a1,0.00,WAKEMED,NC,Subscriber,Female,48,Generation X,743.06,1762.83,...,-271.74,-36.57,-72.42,0.00,2192.41,7835.17,12179.12,12179.12,0.00,4218.01
3,mma-d9fa4b33e14b458992bf0617961ac987,0.00,WAKEMED,NC,Subscriber,Female,53,Generation X,1175.70,4011.46,...,-292.82,-24.91,-223.38,0.00,2196.13,22039.20,15271.61,15271.61,0.00,6565.79
4,mma-a809caefdd8242f3ac89858b2ef2a9c0,0.00,WAKEMED,NC,Subscriber,Female,56,Generation X,6175.82,2379.46,...,-2206.45,-35.73,45.77,936.58,21588.54,17339.17,15705.83,14769.25,1576.93,4005.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,mma-49792ef032014de08dcbe3da819074a0,0.00,CAPITOL BROADCASTING COMPANY INC,NC,Spouse,Female,44,Millenials,927.12,894.17,...,10830.13,1168.14,3325.15,14.15,1672.28,12649.84,7377.63,7363.48,52046.10,3306.80
457,mma-b2ff84367d114503adf5194741af466f,0.00,WAKEMED,NC,Subscriber,Female,66,Baby Boomers,619.58,3593.56,...,2160.60,348.72,1075.11,64.16,6470.59,18271.45,13754.07,13689.91,21335.87,5287.06
458,mma-bbe87a52bbd64d458e04d757809d8531,0.00,SELECT MEDICAL,CT,Subscriber,Female,31,Millenials,272.04,1501.92,...,1063.30,390.87,286.96,105.98,1451.52,10394.69,7289.38,7183.40,6778.14,3049.34
459,mma-8bb9aad32c0741fc9e75241e57690ba9,0.00,TOWN OF APEX,NC,Spouse,Female,42,Millenials,8084.53,4060.86,...,-6395.09,-79.10,-2089.11,46.86,20559.61,17864.01,15832.82,15785.96,33688.43,4466.23


In [24]:
member1 = member_mrg.drop(columns=['MEMBER_STATUS', 'EMPLOYER_NAME'])
member1

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-5a400fb9c1934bcf87c25301710034c3,PA,Spouse,Female,57,Generation X,5783.55,11169.72,10984.51,13666.37,7882.82,136.30,2346.32,63018.44,91263.11,52811.56,69490.32,6471.88,10.27,-1903.59
1,mma-2e25ed887e0b4990ad2fa1c2d91e0966,NC,Subscriber,Female,53,Generation X,32448.94,4684.03,12931.39,6765.57,-25683.37,-79.15,-6880.27,1063.89,8594.32,19333.53,13359.87,12295.98,1155.76,4762.72
2,mma-b027fbc0d58348338e0a12b5520743a1,NC,Subscriber,Female,48,Generation X,743.06,1762.83,1853.81,471.32,-271.74,-36.57,-72.42,0.00,2192.41,7835.17,12179.12,12179.12,0.00,4218.01
3,mma-d9fa4b33e14b458992bf0617961ac987,NC,Subscriber,Female,53,Generation X,1175.70,4011.46,2656.13,882.88,-292.82,-24.91,-223.38,0.00,2196.13,22039.20,15271.61,15271.61,0.00,6565.79
4,mma-a809caefdd8242f3ac89858b2ef2a9c0,NC,Subscriber,Female,56,Generation X,6175.82,2379.46,9456.50,3969.37,-2206.45,-35.73,45.77,936.58,21588.54,17339.17,15705.83,14769.25,1576.93,4005.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,mma-49792ef032014de08dcbe3da819074a0,NC,Spouse,Female,44,Millenials,927.12,894.17,1655.31,11757.25,10830.13,1168.14,3325.15,14.15,1672.28,12649.84,7377.63,7363.48,52046.10,3306.80
457,mma-b2ff84367d114503adf5194741af466f,NC,Subscriber,Female,66,Baby Boomers,619.58,3593.56,7862.89,2780.18,2160.60,348.72,1075.11,64.16,6470.59,18271.45,13754.07,13689.91,21335.87,5287.06
458,mma-bbe87a52bbd64d458e04d757809d8531,CT,Subscriber,Female,31,Millenials,272.04,1501.92,1181.59,1335.34,1063.30,390.87,286.96,105.98,1451.52,10394.69,7289.38,7183.40,6778.14,3049.34
459,mma-8bb9aad32c0741fc9e75241e57690ba9,NC,Spouse,Female,42,Millenials,8084.53,4060.86,2355.05,1689.44,-6395.09,-79.10,-2089.11,46.86,20559.61,17864.01,15832.82,15785.96,33688.43,4466.23


In [25]:
member1.HOME_STATE.value_counts()

HOME_STATE
NC    197
PA     85
TX     29
MI     22
NJ     21
NY     17
GA     12
VA      9
FL      8
KY      7
IL      7
OH      6
CA      5
DE      4
TN      4
KS      4
IN      3
MO      3
MN      3
CO      2
MA      2
WI      2
ID      1
MD      1
OK      1
MS      1
WA      1
NH      1
OR      1
SC      1
CT      1
Name: count, dtype: int64

In [26]:
# Define the states to keep
states_to_keep = ['NC', 'PA', 'TX', 'MI', 'NJ', 'NY', 'GA']
member1['HOME_STATE'] = member1['HOME_STATE'].apply(lambda x: x if x in states_to_keep else 'OTHER')
member1

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-5a400fb9c1934bcf87c25301710034c3,PA,Spouse,Female,57,Generation X,5783.55,11169.72,10984.51,13666.37,7882.82,136.30,2346.32,63018.44,91263.11,52811.56,69490.32,6471.88,10.27,-1903.59
1,mma-2e25ed887e0b4990ad2fa1c2d91e0966,NC,Subscriber,Female,53,Generation X,32448.94,4684.03,12931.39,6765.57,-25683.37,-79.15,-6880.27,1063.89,8594.32,19333.53,13359.87,12295.98,1155.76,4762.72
2,mma-b027fbc0d58348338e0a12b5520743a1,NC,Subscriber,Female,48,Generation X,743.06,1762.83,1853.81,471.32,-271.74,-36.57,-72.42,0.00,2192.41,7835.17,12179.12,12179.12,0.00,4218.01
3,mma-d9fa4b33e14b458992bf0617961ac987,NC,Subscriber,Female,53,Generation X,1175.70,4011.46,2656.13,882.88,-292.82,-24.91,-223.38,0.00,2196.13,22039.20,15271.61,15271.61,0.00,6565.79
4,mma-a809caefdd8242f3ac89858b2ef2a9c0,NC,Subscriber,Female,56,Generation X,6175.82,2379.46,9456.50,3969.37,-2206.45,-35.73,45.77,936.58,21588.54,17339.17,15705.83,14769.25,1576.93,4005.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,mma-49792ef032014de08dcbe3da819074a0,NC,Spouse,Female,44,Millenials,927.12,894.17,1655.31,11757.25,10830.13,1168.14,3325.15,14.15,1672.28,12649.84,7377.63,7363.48,52046.10,3306.80
457,mma-b2ff84367d114503adf5194741af466f,NC,Subscriber,Female,66,Baby Boomers,619.58,3593.56,7862.89,2780.18,2160.60,348.72,1075.11,64.16,6470.59,18271.45,13754.07,13689.91,21335.87,5287.06
458,mma-bbe87a52bbd64d458e04d757809d8531,OTHER,Subscriber,Female,31,Millenials,272.04,1501.92,1181.59,1335.34,1063.30,390.87,286.96,105.98,1451.52,10394.69,7289.38,7183.40,6778.14,3049.34
459,mma-8bb9aad32c0741fc9e75241e57690ba9,NC,Spouse,Female,42,Millenials,8084.53,4060.86,2355.05,1689.44,-6395.09,-79.10,-2089.11,46.86,20559.61,17864.01,15832.82,15785.96,33688.43,4466.23


In [27]:
med.columns

Index(['MEDICAL_CLAIM_ID', 'MEMBER_ID', 'PAID_DATE', 'PAID_YEAR',
       'SERVICE_DATE', 'SERVICE_YEAR', 'MEDICAL_PAID_AMOUNT',
       'IS_TELEMEDICINE', 'IS_ER_AVOIDABLE', 'PRIMARY_DX', 'PROCEDURE_DESC',
       'ARTTOS_V2_L1', 'ARTTOS_V2_L3', 'DX_IS_CHRONIC', 'ICD10_CHAPTER',
       'ICD10_CATEGORY', 'ICD10_SECTION', 'MEG_EPISODE_DESCRIPTION',
       'EMPLOYER_NAME', 'OP_SURG_INC', 'IS_PCP_VISIT', 'INCLUDED_SPECIALIST',
       'ER_VISIT_FLAG', 'IS_URGENT_CARE_VISIT', 'IS_PREVENTIVE_VISIT',
       'IP_ADMIT_INC'],
      dtype='object')

In [28]:
med_util = med.groupby('MEMBER_ID').agg({
    'DX_IS_CHRONIC': 'max',
    'ER_VISIT_FLAG': 'sum',
    'IS_PCP_VISIT': 'sum',
    'IS_URGENT_CARE_VISIT': 'sum',
    'IS_PREVENTIVE_VISIT': 'sum',
    'IP_ADMIT_INC': 'sum'
}).reset_index()
med_util

,MEMBER_ID,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,mma-0049f2985b9f413a8cf5ad2f79c95d4b,1.00,0,8,0,5,0
1,mma-0054034112c64cd2924c4f85de9a46ca,1.00,0,8,1,4,0
2,mma-010b8797194145fbb3687e60959516b1,1.00,2,2,1,0,0
3,mma-01c846aa858d43e98167eb0669db3a4e,1.00,0,6,1,6,0
4,mma-0281be178d4042ab83dd30b3190a3c6c,1.00,43,18,9,5,0
...,...,...,...,...,...,...,...
721,mma-fe17629a213c48c299541abcf3098a08,1.00,4,60,0,2,0
722,mma-fe87330ad1d34f5bb4ee0bba91cf61c7,1.00,0,4,0,1,0
723,mma-ff1781243560419d986a286a5cae8c89,1.00,2,6,4,3,0
724,mma-ff82ab56bc48478b91ad9760b62d56e1,1.00,13,16,17,3,0


In [29]:
episodes = med.MEG_EPISODE_DESCRIPTION.value_counts()
episodes.head(10)

MEG_EPISODE_DESCRIPTION
Encounter for Preventive Health Services              19650
Overweight and Obesity                                11625
Essential Hypertension, Chronic Maintenance            8879
Osteoarthritis, Except Spine                           7044
Other Arthropathies, Bone and Joint Disorders          5717
Depression                                             5391
Diabetes Mellitus Type 2 & Unspec Type Maintenance     4220
Other Spinal and Back Disorders, Low Back              4193
Hypothyroidism                                         3768
Factors Influencing Health Status                      3699
Name: count, dtype: int64

In [30]:
episodes_cost = med.groupby('MEG_EPISODE_DESCRIPTION')['MEDICAL_PAID_AMOUNT'].sum().reset_index()
episodes_cost.sort_values(by='MEDICAL_PAID_AMOUNT', ascending=False)

,MEG_EPISODE_DESCRIPTION,MEDICAL_PAID_AMOUNT
110,Encounter for Preventive Health Services,1855685.32
245,"Osteoarthritis, Except Spine",1354076.98
298,Overweight and Obesity,1033283.39
116,"Essential Hypertension, Chronic Maintenance",649166.51
195,Multiple Sclerosis,599093.18
...,...,...
131,"Fracture: Humerus, Supracondylar",0.00
145,"Gonorrhea, Male",0.00
273,Other Electrolyte Disorders,0.00
231,"Neoplasm, Malignant: Unspecified Primary Site",0.00


In [31]:
episodes_to_keep = ['Encounter for Preventive Health Services', 'Overweight and Obesity', 'Essential Hypertension, Chronic Maintenance', 
                    'Osteoarthritis, Except Spine', 'Other Arthropathies, Bone and Joint Disorders', 'Depression', 'Diabetes Mellitus Type 2 & Unspec Type Maintenance',
                   'Other Spinal and Back Disorders, Low Back', 'Hypothyroidism', 'Factors Influencing Health Status']
med1 = med
med1['MEG_EPISODE_DESCRIPTION'] = med['MEG_EPISODE_DESCRIPTION'].apply(lambda x: x if x in episodes_to_keep else 'OTHER')

In [32]:
med_meg = pd.pivot_table(
    med1,
    values='MEDICAL_PAID_AMOUNT',
    index='MEMBER_ID',
    columns='MEG_EPISODE_DESCRIPTION',
    aggfunc='count',
    fill_value=0
)
med_meg.reset_index()
med_meg.columns.name = None
med_meg.drop(columns='OTHER', inplace=True)
med_meg

,Depression,Diabetes Mellitus Type 2 & Unspec Type Maintenance,Encounter for Preventive Health Services,"Essential Hypertension, Chronic Maintenance",Factors Influencing Health Status,Hypothyroidism,"Osteoarthritis, Except Spine","Other Arthropathies, Bone and Joint Disorders","Other Spinal and Back Disorders, Low Back",Overweight and Obesity
MEMBER_ID,,,,,,,,,,
mma-0049f2985b9f413a8cf5ad2f79c95d4b,0,14,11,17,4,0,3,3,0,0
mma-0054034112c64cd2924c4f85de9a46ca,0,0,37,44,0,0,0,0,32,0
mma-010b8797194145fbb3687e60959516b1,1,0,0,0,0,2,0,0,0,0
mma-01c846aa858d43e98167eb0669db3a4e,0,0,16,0,0,0,0,0,0,11
mma-0281be178d4042ab83dd30b3190a3c6c,0,0,24,0,0,0,10,12,0,8
...,...,...,...,...,...,...,...,...,...,...
mma-fe17629a213c48c299541abcf3098a08,66,34,31,72,73,0,90,94,2,104
mma-fe87330ad1d34f5bb4ee0bba91cf61c7,0,0,5,20,0,0,0,0,0,0
mma-ff1781243560419d986a286a5cae8c89,0,1,57,14,0,0,0,0,0,5


In [33]:
med_var = member1.merge(med_util, on='MEMBER_ID')
med_var = med_var.merge(med_meg, on='MEMBER_ID')
med_var.columns= med_var.columns.str.replace(' ', '_')
med_var

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,...,Depression,Diabetes_Mellitus_Type_2_&_Unspec_Type_Maintenance,Encounter_for_Preventive_Health_Services,"Essential_Hypertension,_Chronic_Maintenance",Factors_Influencing_Health_Status,Hypothyroidism,"Osteoarthritis,_Except_Spine","Other_Arthropathies,_Bone_and_Joint_Disorders","Other_Spinal_and_Back_Disorders,_Low_Back",Overweight_and_Obesity
0,mma-5a400fb9c1934bcf87c25301710034c3,PA,Spouse,Female,57,Generation X,5783.55,11169.72,10984.51,13666.37,...,1,1,30,0,27,57,4,0,13,36
1,mma-2e25ed887e0b4990ad2fa1c2d91e0966,NC,Subscriber,Female,53,Generation X,32448.94,4684.03,12931.39,6765.57,...,0,0,25,43,7,0,58,93,0,66
2,mma-b027fbc0d58348338e0a12b5520743a1,NC,Subscriber,Female,48,Generation X,743.06,1762.83,1853.81,471.32,...,0,0,11,0,0,0,0,0,0,21
3,mma-d9fa4b33e14b458992bf0617961ac987,NC,Subscriber,Female,53,Generation X,1175.70,4011.46,2656.13,882.88,...,4,3,16,41,0,0,0,4,0,16
4,mma-a809caefdd8242f3ac89858b2ef2a9c0,NC,Subscriber,Female,56,Generation X,6175.82,2379.46,9456.50,3969.37,...,0,26,15,0,16,0,30,2,3,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,mma-49792ef032014de08dcbe3da819074a0,NC,Spouse,Female,44,Millenials,927.12,894.17,1655.31,11757.25,...,0,2,38,0,0,0,0,6,0,1
457,mma-b2ff84367d114503adf5194741af466f,NC,Subscriber,Female,66,Baby Boomers,619.58,3593.56,7862.89,2780.18,...,0,0,50,22,0,0,0,0,0,23
458,mma-bbe87a52bbd64d458e04d757809d8531,OTHER,Subscriber,Female,31,Millenials,272.04,1501.92,1181.59,1335.34,...,0,6,14,0,0,0,0,0,0,40
459,mma-8bb9aad32c0741fc9e75241e57690ba9,NC,Spouse,Female,42,Millenials,8084.53,4060.86,2355.05,1689.44,...,1,19,28,0,0,0,0,5,0,4


In [45]:
med_var.columns

Index(['MEMBER_ID', 'HOME_STATE', 'MEMBER_RELATIONSHIP', 'GENDER', 'Age',
       'GENERATIONS', 'Med_2021', 'Med_2022', 'Med_2023', 'Med_2024',
       'Med_Delta', 'Med_%_Change', 'Med_Slope', 'Rx_2021', 'Rx_2022',
       'Rx_2023', 'Rx_2024', 'Rx_Delta', 'Rx_%_Change', 'Rx_Slope',
       'DX_IS_CHRONIC', 'ER_VISIT_FLAG', 'IS_PCP_VISIT',
       'IS_URGENT_CARE_VISIT', 'IS_PREVENTIVE_VISIT', 'IP_ADMIT_INC',
       'Depression', 'Diabetes_Mellitus_Type_2_&_Unspec_Type_Maintenance',
       'Encounter_for_Preventive_Health_Services',
       'Essential_Hypertension,_Chronic_Maintenance',
       'Factors_Influencing_Health_Status', 'Hypothyroidism',
       'Osteoarthritis,_Except_Spine',
       'Other_Arthropathies,_Bone_and_Joint_Disorders',
       'Other_Spinal_and_Back_Disorders,_Low_Back', 'Overweight_and_Obesity'],
      dtype='object')

In [49]:
from statsmodels.formula.api import ols
model1 = ols("""Med_Delta ~ Depression""", data=med_var).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Med_Delta   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.1652
Date:                Wed, 16 Apr 2025   Prob (F-statistic):              0.685
Time:                        13:21:50   Log-Likelihood:                -5538.7
No. Observations:                 461   AIC:                         1.108e+04
Df Residuals:                     459   BIC:                         1.109e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1338.9351   1938.900     -0.691      0.490   -5149.156    2471.286
Depression   -36.4058     89.571     -0.406      0.685    -212.426     139.614
==============================================================================
Omnibus:                      523.094   Durbin-Watson:                   2.029
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           204384.294
Skew:                           4.451   Prob(JB):                         0.00
Kurtosis:                     105.768   Cond. No.                         22.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
model2 = ols("""Med_Slope ~ Age""", data=med_var).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Med_Slope   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.1668
Date:                Wed, 16 Apr 2025   Prob (F-statistic):              0.683
Time:                        10:23:00   Log-Likelihood:                -4993.8
No. Observations:                 461   AIC:                             9992.
Df Residuals:                     459   BIC:                         1.000e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1613.5849   2992.091     -0.539      0.590   -7493.480    4266.310
Age           24.4574     59.886      0.408      0.683     -93.228     142.143
==============================================================================
Omnibus:                      491.555   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           169033.611
Skew:                           3.992   Prob(JB):                         0.00
Kurtosis:                      96.468   Cond. No.                         261.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [43]:
model3 = ols("""Med_Delta ~ Encounter_for_Preventive_Health_Services""", data=med_var).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Med_Delta   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                   0.09798
Date:                Wed, 16 Apr 2025   Prob (F-statistic):              0.754
Time:                        13:15:31   Log-Likelihood:                -5538.8
No. Observations:                 461   AIC:                         1.108e+04
Df Residuals:                     459   BIC:                         1.109e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                 -680.3195   3357.282     -0.203      0.840   -7277.868    5917.229
Encounter_for_Preventive_Health_Services   -34.0425    108.757     -0.313      0.754    -247.766     179.681
==============================================================================
Omnibus:                      521.248   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           204460.573
Skew:                           4.419   Prob(JB):                         0.00
Kurtosis:                     105.792   Cond. No.                         55.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [39]:
med_var.describe()

,Age,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,...,Depression,Diabetes_Mellitus_Type_2_&_Unspec_Type_Maintenance,Encounter_for_Preventive_Health_Services,"Essential_Hypertension,_Chronic_Maintenance",Factors_Influencing_Health_Status,Hypothyroidism,"Osteoarthritis,_Except_Spine","Other_Arthropathies,_Bone_and_Joint_Disorders","Other_Spinal_and_Back_Disorders,_Low_Back",Overweight_and_Obesity
count,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,...,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00
mean,49.04,10785.83,8428.90,8103.09,9513.89,-1554.05,inf,-414.16,4030.60,12071.98,...,5.91,4.36,25.67,11.96,4.00,4.57,8.12,7.90,4.79,15.64
std,9.56,28876.91,17172.10,15900.76,30679.53,40012.57,NaN,12270.73,16040.31,18195.96,...,20.85,10.76,17.17,21.59,9.86,13.42,30.62,21.45,19.91,19.76
min,19.00,-6733.78,-2675.94,0.00,-3113.33,-351945.16,-218.99,-106061.39,-35797.51,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,43.00,927.12,1266.58,1105.60,1153.71,-2486.06,-60.81,-968.58,82.82,3229.66,...,0.00,0.00,13.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00
50%,49.00,2591.38,3068.68,2975.91,2912.37,152.06,7.08,31.01,353.31,7369.97,...,0.00,0.00,22.00,0.00,0.00,0.00,0.00,1.00,0.00,9.00
75%,56.00,7357.73,7854.88,7862.89,8883.64,2871.99,234.71,1322.50,1281.63,15020.11,...,0.00,3.00,35.00,17.00,3.00,0.00,0.00,6.00,1.00,23.00
max,75.00,357120.96,222048.77,186327.16,566726.05,565812.16,inf,169296.90,201096.74,223585.97,...,172.00,95.00,116.00,146.00,90.00,97.00,273.00,218.00,293.00,140.00


In [40]:
med_var.columns

Index(['MEMBER_ID', 'HOME_STATE', 'MEMBER_RELATIONSHIP', 'GENDER', 'Age',
       'GENERATIONS', 'Med_2021', 'Med_2022', 'Med_2023', 'Med_2024',
       'Med_Delta', 'Med_%_Change', 'Med_Slope', 'Rx_2021', 'Rx_2022',
       'Rx_2023', 'Rx_2024', 'Rx_Delta', 'Rx_%_Change', 'Rx_Slope',
       'DX_IS_CHRONIC', 'ER_VISIT_FLAG', 'IS_PCP_VISIT',
       'IS_URGENT_CARE_VISIT', 'IS_PREVENTIVE_VISIT', 'IP_ADMIT_INC',
       'Depression', 'Diabetes_Mellitus_Type_2_&_Unspec_Type_Maintenance',
       'Encounter_for_Preventive_Health_Services',
       'Essential_Hypertension,_Chronic_Maintenance',
       'Factors_Influencing_Health_Status', 'Hypothyroidism',
       'Osteoarthritis,_Except_Spine',
       'Other_Arthropathies,_Bone_and_Joint_Disorders',
       'Other_Spinal_and_Back_Disorders,_Low_Back', 'Overweight_and_Obesity'],
      dtype='object')

In [41]:
costs = med_var[['MEMBER_ID', 'HOME_STATE', 'MEMBER_RELATIONSHIP', 'GENDER', 'Age',
       'GENERATIONS', 'Med_2021', 'Med_2022', 'Med_2023', 'Med_2024',
       'Med_Delta', 'Med_%_Change', 'Med_Slope', 'Rx_2021', 'Rx_2022',
       'Rx_2023', 'Rx_2024', 'Rx_Delta', 'Rx_%_Change', 'Rx_Slope']]
costs

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,mma-5a400fb9c1934bcf87c25301710034c3,PA,Spouse,Female,57,Generation X,5783.55,11169.72,10984.51,13666.37,7882.82,136.30,2346.32,63018.44,91263.11,52811.56,69490.32,6471.88,10.27,-1903.59
1,mma-2e25ed887e0b4990ad2fa1c2d91e0966,NC,Subscriber,Female,53,Generation X,32448.94,4684.03,12931.39,6765.57,-25683.37,-79.15,-6880.27,1063.89,8594.32,19333.53,13359.87,12295.98,1155.76,4762.72
2,mma-b027fbc0d58348338e0a12b5520743a1,NC,Subscriber,Female,48,Generation X,743.06,1762.83,1853.81,471.32,-271.74,-36.57,-72.42,0.00,2192.41,7835.17,12179.12,12179.12,0.00,4218.01
3,mma-d9fa4b33e14b458992bf0617961ac987,NC,Subscriber,Female,53,Generation X,1175.70,4011.46,2656.13,882.88,-292.82,-24.91,-223.38,0.00,2196.13,22039.20,15271.61,15271.61,0.00,6565.79
4,mma-a809caefdd8242f3ac89858b2ef2a9c0,NC,Subscriber,Female,56,Generation X,6175.82,2379.46,9456.50,3969.37,-2206.45,-35.73,45.77,936.58,21588.54,17339.17,15705.83,14769.25,1576.93,4005.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,mma-49792ef032014de08dcbe3da819074a0,NC,Spouse,Female,44,Millenials,927.12,894.17,1655.31,11757.25,10830.13,1168.14,3325.15,14.15,1672.28,12649.84,7377.63,7363.48,52046.10,3306.80
457,mma-b2ff84367d114503adf5194741af466f,NC,Subscriber,Female,66,Baby Boomers,619.58,3593.56,7862.89,2780.18,2160.60,348.72,1075.11,64.16,6470.59,18271.45,13754.07,13689.91,21335.87,5287.06
458,mma-bbe87a52bbd64d458e04d757809d8531,OTHER,Subscriber,Female,31,Millenials,272.04,1501.92,1181.59,1335.34,1063.30,390.87,286.96,105.98,1451.52,10394.69,7289.38,7183.40,6778.14,3049.34
459,mma-8bb9aad32c0741fc9e75241e57690ba9,NC,Spouse,Female,42,Millenials,8084.53,4060.86,2355.05,1689.44,-6395.09,-79.10,-2089.11,46.86,20559.61,17864.01,15832.82,15785.96,33688.43,4466.23


In [42]:
costs.describe()

,Age,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
count,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00,461.00
mean,49.04,10785.83,8428.90,8103.09,9513.89,-1554.05,inf,-414.16,4030.60,12071.98,17114.74,15865.22,11834.62,13919.00,4054.66
std,9.56,28876.91,17172.10,15900.76,30679.53,40012.57,NaN,12270.73,16040.31,18195.96,17275.50,18398.69,18506.06,87465.91,6022.53
min,19.00,-6733.78,-2675.94,0.00,-3113.33,-351945.16,-218.99,-106061.39,-35797.51,0.00,39.83,307.20,-178299.60,-146.54,-48086.39
25%,43.00,927.12,1266.58,1105.60,1153.71,-2486.06,-60.81,-968.58,82.82,3229.66,9224.75,8872.36,6904.86,234.86,2328.96
50%,49.00,2591.38,3068.68,2975.91,2912.37,152.06,7.08,31.01,353.31,7369.97,15280.25,12927.53,11249.63,1821.01,3890.98
75%,56.00,7357.73,7854.88,7862.89,8883.64,2871.99,234.71,1322.50,1281.63,15020.11,18511.48,16237.32,14922.30,6774.62,5168.75
max,75.00,357120.96,222048.77,186327.16,566726.05,565812.16,inf,169296.90,201096.74,223585.97,170115.73,206102.47,203204.41,1732629.77,70188.75


average med+rx net per member, normalized for 2024 = $11,834-1,554 = $10,280. Since 2021, employers have spent $10,280 average per member on those that have prescriptions for Wegovy, Saxenda or Zepbound for 3 years

In [52]:
med_var.to_csv('glp1_members.csv', index=False)

In [53]:
drugs = rx.groupby(['ART_DRUGGRPS_L1', 'ART_DRUGGRPS_L2', 'DRUG_NAME_PREFERRED'])['RX_PAID_AMOUNT'].sum().reset_index()
drugs.sort_values(by='RX_PAID_AMOUNT', ascending=False, inplace=True)
drugs

,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT
303,CNS Agents,Weight Loss Agents,Wegovy,17251767.28
302,CNS Agents,Weight Loss Agents,Saxenda,4358296.11
305,CNS Agents,Weight Loss Agents,Zepbound,1593951.74
86,CNS Agents,Anti-Rheumatics/Immune Modulators,Humira (2 Pen),1133602.61
1076,Topical Agents,Psoriasis Agents,Stelara,1072764.97
...,...,...,...,...
465,Endocrine,Thyroid Agents,Nature-Throid,-0.62
213,CNS Agents,NSAID Analgesics,Duexis,-20.70
528,Gastrointestinal,Proton Pum Inhibitors,Omeprazole-Sodium Bicarbonate,-5403.49
221,CNS Agents,NSAID Analgesics,Ketoprofen,-7348.34
